In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from pathlib import Path
import pandas as pd
import nbimporter
from data_preprocessing import DATA_DIR

import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import TimeDistributed, Dense, Input, GlobalAveragePooling2D ,BatchNormalization ,Flatten ,Bidirectional , GRU , Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger ,ReduceLROnPlateau

In [ ]:
# Load a sequence of 15 image frames from a sample folder
def load_sequence_frames(sample_path, seq_len=15, step = 2, size=(224, 224)):
    frames = []
    image_files = sorted(sample_path.glob('*.png'))

    # Take 15 frames with a stride of 2 (i.e., pick every 2nd frame)
    selected_files = image_files[::step][:seq_len]

    for img_path in selected_files:
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, size)
        frames.append(img)

    frames = np.array(frames, dtype=np.float32) / 255.0
    return frames  # Shape: (15, 224, 224, 3)

# Dataset generator
def data_generator(root_dir):
    root = Path(root_dir)
    classes = sorted([d.name for d in root.iterdir() if d.is_dir()])
    print(classes)
    class_to_idx = {c: i for i, c in enumerate(classes)}

    samples, labels = [], []
    for c in classes:
        for sample_folder in (root / c).iterdir():
            if sample_folder.is_dir():
                samples.append(sample_folder)
                labels.append(class_to_idx[c])

    def gen():
        for sample_path, label in zip(samples, labels):
            seq = load_sequence_frames(sample_path)
            yield seq, label

    return tf.data.Dataset.from_generator(
        gen,
        output_signature=(
            tf.TensorSpec(shape=(15, 224, 224, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.int32)
        )
    )

In [ ]:
# No augmentation applied here
train_dataset = data_generator(f"{DATA_DIR}/train")\
    .shuffle(100)\
    .batch(4)\
    .repeat()\
    .prefetch(tf.data.AUTOTUNE)

val_dataset = data_generator(f"{DATA_DIR}/val")\
    .batch(4)\
    .prefetch(tf.data.AUTOTUNE)

In [ ]:
num_classes = 5
sequence_length = 15
image_size = (224, 224, 3)

# Input: sequence of images
inputs = Input(shape=(sequence_length, *image_size))

# Use MobileNet as CNN feature extractor (weights pretrained on ImageNet)
cnn_base = MobileNetV2(include_top=False, weights='imagenet', input_shape=image_size)

# unFreeze CNN base initially
cnn_base.trainable = True

# Apply CNN to each frame separately via TimeDistributed
x = TimeDistributed(cnn_base)(inputs)
x = TimeDistributed(BatchNormalization())(x)
x = TimeDistributed(GlobalAveragePooling2D())(x)

# Apply a GRU layer with 128 units to capture temporal dependencies across frames,
# with L2 regularization (weight decay) to reduce overfitting
x = x = Bidirectional(GRU(128), kernel_regularizer=l2(1e-4) )(x)
x = Dropout(0.25)(x)

x = Dense(128, activation='relu' , kernel_regularizer=l2(1e-4) )(x)
x = Dropout(0.25)(x)

# Output layer
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
original_train_dataset = data_generator(f"{DATA_DIR}/train")
num_train_samples = sum(1 for _ in original_train_dataset)

original_val_dataset = data_generator(f"{DATA_DIR}/val")
num_val_samples = sum(1 for _ in original_val_dataset)

steps_per_epoch = num_train_samples // 4
validation_steps = num_val_samples // 4

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

checkpoint = ModelCheckpoint(f"{DATA_DIR}/Outputs/best_model.keras", save_best_only=True, monitor='val_accuracy')

csv_logger = CSVLogger(f"{DATA_DIR}/Outputs/training_log.csv", append=False)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',    # metric to monitor
    factor=0.5,            # factor to reduce LR by, new_lr = lr * factor
    patience=3,            # number of epochs with no improvement before reducing LR
    verbose=1,
    min_lr=1e-7            # lower bound on LR
)

history = model.fit(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_dataset,
    validation_steps=validation_steps,
    epochs=40,
    callbacks=[early_stop, checkpoint , csv_logger , reduce_lr]
)

In [ ]:
# Load the saved history from file
history = pd.read_csv(f"{DATA_DIR}/Outputs/training_log.csv")

plt.figure(figsize=(14, 5))

# Plot Accuracy
plt.subplot(1, 2, 1)
plt.plot(history['accuracy'], label='Train Accuracy', marker='o')
plt.plot(history['val_accuracy'], label='Val Accuracy', marker='o')
plt.title('Training vs Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(history['loss'], label='Train Loss', marker='o')
plt.plot(history['val_loss'], label='Val Loss', marker='o')
plt.title('Training vs Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()